# 입문자를 위한, 파이썬/R 데이터 분석      

]

##  (공공데이터분석) 국토부 아파트매매 실거래가 API[data.go.kr] 연동

#### XML 국토교통부_아파트매매 실거래자료 : https://www.data.go.kr/data/15058747/openapi.do

#### 행정표준관리시스템 : https://www.code.go.kr/index.do ( [메뉴] 자주이용하는 코드 > 법정동 > [법정동코드전체자료] )

]

### 1. Import Packages

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

### 2. Data Loading 

#### : API Key Setting

In [2]:
f=open("data/my_api_key.txt","r")
lines=f.readlines()
my_key=lines[0]
f.close()

In [3]:
my_key

'c%2B9O5FYgyH4YJA4VAeems4Nhk34TQIIAYm1ocLLGy%2FrG02SYK2k8AU0QBUgtn2q2CZC4CDxYkay1aM9GQtw6wg%3D%3D'

#### : Pre-defined

In [4]:
#"http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade
#        ?
#        LAWD_CD=11110
#        &
#        DEAL_YMD=201512
#        &
#        serviceKey=서비스키"

In [5]:
base_url = "http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade"

In [6]:
area_cd = "LAWD_CD=" + "11680"    # 11680 : 강남구
deal_dt = "DEAL_YMD=" + "202404"
svc_key = "serviceKey=" + my_key

In [7]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11680&DEAL_YMD=202404&serviceKey=c%2B9O5FYgyH4YJA4VAeems4Nhk34TQIIAYm1ocLLGy%2FrG02SYK2k8AU0QBUgtn2q2CZC4CDxYkay1aM9GQtw6wg%3D%3D'

#### : Request Test

In [8]:
response = requests.get(url)
response

<Response [200]>

In [9]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><거래금액>     245,000</거래금액><거래유형>중개거래</거래유형><건축년도>2006</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>역삼푸르지오</아파트><월>4</월><일>2</일><전용면적>84.9097</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>754-1</지번><지역코드>11680</지역코드><층>16</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     268,000</거래금액><거래유형>중개거래</거래유형><건축년도>2012</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>개나리SK뷰</아파트><월>4</월><일>5</일><전용면적>84.99</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>716-3</지번><지역코드>11680</지역코드><층>4</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     196,000</거래금액><거래유형>중개거래</거래유형><건축년도>2005</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>e-편한세상</아파트><월>4</월><일>5</일><전용면적>59.606</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>755-4</지번><지역코드>11680</지역코드><층

### Addtional Preparation : 

#### -1)  Input Data : lawd_cd / 법정동 코드(지역코드) 가공

In [10]:
lawd_cd = pd.read_csv("data/법정동코드 전체자료.txt", sep='\t', encoding ='cp949') #'euc-kr', 'utf-8'
lawd_cd

,법정동코드,법정동명,폐지여부
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
49845,5280042024,전북특별자치도 부안군 위도면 대리,존재
49846,5280042025,전북특별자치도 부안군 위도면 거륜리,존재
49847,5280042026,전북특별자치도 부안군 위도면 식도리,존재
49848,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재


In [11]:
# Column 이름 변경 : 데이터 조작 편의성을 위함
lawd_cd.columns

Index(['법정동코드', '법정동명', '폐지여부'], dtype='object')

In [12]:
lawd_cd.columns = ["code", "name", "check"]
lawd_cd

,code,name,check
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재
...,...,...,...
49845,5280042024,전북특별자치도 부안군 위도면 대리,존재
49846,5280042025,전북특별자치도 부안군 위도면 거륜리,존재
49847,5280042026,전북특별자치도 부안군 위도면 식도리,존재
49848,5280042027,전북특별자치도 부안군 위도면 상왕등리,존재


In [13]:
lawd_cd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49850 entries, 0 to 49849
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   code    49850 non-null  int64 
 1   name    49850 non-null  object
 2   check   49850 non-null  object
dtypes: int64(1), object(2)
memory usage: 1.1+ MB


In [14]:
in_name = "강남구"

In [15]:
lawd_cd['name'].str.contains(in_name)

0        False
1        False
2        False
3        False
4        False
         ...  
49845    False
49846    False
49847    False
49848    False
49849    False
Name: name, Length: 49850, dtype: bool

In [16]:
lawd_cd['check'] == "존재"

0        True
1        True
2        True
3        True
4        True
         ... 
49845    True
49846    True
49847    True
49848    True
49849    True
Name: check, Length: 49850, dtype: bool

In [17]:
(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")

0        False
1        False
2        False
3        False
4        False
         ...  
49845    False
49846    False
49847    False
49848    False
49849    False
Length: 49850, dtype: bool

In [18]:
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")]

,code,name,check
1003,1168000000,서울특별시 강남구,존재
1037,1168010100,서울특별시 강남구 역삼동,존재
1039,1168010300,서울특별시 강남구 개포동,존재
1040,1168010400,서울특별시 강남구 청담동,존재
1041,1168010500,서울특별시 강남구 삼성동,존재
1042,1168010600,서울특별시 강남구 대치동,존재
1043,1168010700,서울특별시 강남구 신사동,존재
1044,1168010800,서울특별시 강남구 논현동,존재
1045,1168011000,서울특별시 강남구 압구정동,존재
1046,1168011100,서울특별시 강남구 세곡동,존재


In [19]:
tmp = lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1)
tmp

,code,name,check
1003,1168000000,서울특별시 강남구,존재


In [20]:
tmp.loc[1003,'code']    # 가져오기는 하나 문제

1168000000

In [21]:
tmp.iloc[0,0]           # 인덱스로 데이터를 가져오도록

1168000000

In [22]:
tmp = tmp.iloc[0,0]
print(tmp)
type(tmp)

1168000000


numpy.int64

In [23]:
tmp

1168000000

In [24]:
str(tmp)

'1168000000'

In [25]:
tmp.astype(str)

'1168000000'

In [26]:
tmp = tmp.astype(str)
tmp

'1168000000'

In [27]:
tmp[0:5]

'11680'

In [28]:
lawd_cd[(lawd_cd['name'].str.contains(in_name)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]

'11680'

In [29]:
# Function 작성 :
def find_lawd_cd(in_text):
    result = lawd_cd[(lawd_cd['name'].str.contains(in_text)) & (lawd_cd['check'] == "존재")].head(1).iloc[0,0].astype(str)[0:5]    
    return result

In [30]:
in_name = "강남구"
find_lawd_cd(in_name)

'11680'

#### -2)  Input Data : lawd_cd, deal_ymd 입력받기

In [31]:
in_lawd = input("검색지역 입력 : ")

검색지역 입력 : 강남구


In [32]:
in_lawd

'강남구'

In [33]:
in_deal_ymd = input("계약월 입력(예: 201912) : ")

계약월 입력(예: 201912) : 202403


In [34]:
in_deal_ymd

'202403'

In [35]:
in_svckey = my_key

In [36]:
in_lawd_cd = find_lawd_cd(in_lawd)
in_lawd_cd

'11680'

In [37]:
area_cd = "LAWD_CD=" + in_lawd_cd
deal_dt = "DEAL_YMD=" + in_deal_ymd
svc_key = "serviceKey=" + in_svckey

In [38]:
url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key
url

'http://openapi.molit.go.kr:8081/OpenAPI_ToolInstallPackage/service/rest/RTMSOBJSvc/getRTMSDataSvcAptTrade?LAWD_CD=11680&DEAL_YMD=202403&serviceKey=c%2B9O5FYgyH4YJA4VAeems4Nhk34TQIIAYm1ocLLGy%2FrG02SYK2k8AU0QBUgtn2q2CZC4CDxYkay1aM9GQtw6wg%3D%3D'

In [39]:
response = requests.get(url)
response

<Response [200]>

In [40]:
response.text

'<?xml version="1.0" encoding="UTF-8" standalone="yes"?><response><header><resultCode>00</resultCode><resultMsg>NORMAL SERVICE.</resultMsg></header><body><items><item><거래금액>     197,000</거래금액><거래유형>중개거래</거래유형><건축년도>2005</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>e-편한세상</아파트><월>3</월><일>3</일><전용면적>59.606</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>755-4</지번><지역코드>11680</지역코드><층>6</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>      61,000</거래금액><거래유형>직거래</거래유형><건축년도>2003</건축년도><년>2024</년><동> </동><등기일자>24.04.03</등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>노블스위트</아파트><월>3</월><일>5</일><전용면적>78.94</전용면적><중개사소재지> </중개사소재지><지번>747</지번><지역코드>11680</지역코드><층>4</층><해제사유발생일> </해제사유발생일><해제여부> </해제여부></item><item><거래금액>     198,000</거래금액><거래유형>중개거래</거래유형><건축년도>2005</건축년도><년>2024</년><동> </동><등기일자> </등기일자><매도자>개인</매도자><매수자>개인</매수자><법정동> 역삼동</법정동><아파트>e-편한세상</아파트><월>3</월><일>6</일><전용면적>59.606</전용면적><중개사소재지>서울 강남구</중개사소재지><지번>755-4</지번><지역코드>11680</지역코드><층>8</

### 3. Data Preprocessng 

#### : web data / API output

In [41]:
soup = BeautifulSoup(response.text, "lxml")
soup

C:\Users\EZEN\anaconda3\lib\site-packages\bs4\builder\__init__.py:545: XMLParsedAsHTMLWarning: It looks like you're parsing an XML document using an HTML parser. If this really is an HTML document (maybe it's XHTML?), you can ignore or filter this warning. If it's XML, you should know that using an XML parser will be more reliable. To parse this document as XML, make sure you have the lxml package installed, and pass the keyword argument `features="xml"` into the BeautifulSoup constructor.
  warnings.warn(


<?xml version="1.0" encoding="UTF-8" standalone="yes"?><html><body><response><header><resultcode>00</resultcode><resultmsg>NORMAL SERVICE.</resultmsg></header><items><item>&lt;거래금액&gt;     197,000거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2005건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;e-편한세상아파트&gt;&lt;월&gt;3월&gt;&lt;일&gt;3일&gt;&lt;전용면적&gt;59.606전용면적&gt;&lt;중개사소재지&gt;서울 강남구중개사소재지&gt;&lt;지번&gt;755-4지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;6층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;</item><item>&lt;거래금액&gt;      61,000거래금액&gt;&lt;거래유형&gt;직거래거래유형&gt;&lt;건축년도&gt;2003건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt;24.04.03등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;노블스위트아파트&gt;&lt;월&gt;3월&gt;&lt;일&gt;5일&gt;&lt;전용면적&gt;78.94전용면적&gt;&lt;중개사소재지&gt; 중개사소재지&gt;&lt;지번&gt;747지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;4층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&

In [42]:
soup.find_all('item')

[<item>&lt;거래금액&gt;     197,000거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2005건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;e-편한세상아파트&gt;&lt;월&gt;3월&gt;&lt;일&gt;3일&gt;&lt;전용면적&gt;59.606전용면적&gt;&lt;중개사소재지&gt;서울 강남구중개사소재지&gt;&lt;지번&gt;755-4지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;6층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;</item>,
 <item>&lt;거래금액&gt;      61,000거래금액&gt;&lt;거래유형&gt;직거래거래유형&gt;&lt;건축년도&gt;2003건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt;24.04.03등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;노블스위트아파트&gt;&lt;월&gt;3월&gt;&lt;일&gt;5일&gt;&lt;전용면적&gt;78.94전용면적&gt;&lt;중개사소재지&gt; 중개사소재지&gt;&lt;지번&gt;747지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;4층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;</item>,
 <item>&lt;거래금액&gt;     198,000거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2005건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;

In [43]:
item = soup.find('item')
item

<item>&lt;거래금액&gt;     197,000거래금액&gt;&lt;거래유형&gt;중개거래거래유형&gt;&lt;건축년도&gt;2005건축년도&gt;&lt;년&gt;2024년&gt;&lt;동&gt; 동&gt;&lt;등기일자&gt; 등기일자&gt;&lt;매도자&gt;개인매도자&gt;&lt;매수자&gt;개인매수자&gt;&lt;법정동&gt; 역삼동법정동&gt;&lt;아파트&gt;e-편한세상아파트&gt;&lt;월&gt;3월&gt;&lt;일&gt;3일&gt;&lt;전용면적&gt;59.606전용면적&gt;&lt;중개사소재지&gt;서울 강남구중개사소재지&gt;&lt;지번&gt;755-4지번&gt;&lt;지역코드&gt;11680지역코드&gt;&lt;층&gt;6층&gt;&lt;해제사유발생일&gt; 해제사유발생일&gt;&lt;해제여부&gt; 해제여부&gt;</item>

In [44]:
#item.find('거래금액')#.text

In [45]:
import xml.etree.ElementTree as ET

In [46]:
response#.content#.text

<Response [200]>

In [47]:
root = ET.fromstring(response.content)
root

<Element 'response' at 0x00000161C4B13450>

In [48]:
item_list = []

In [49]:
root.find('body').find('items')[0]

<Element 'item' at 0x00000161C4B13630>

In [50]:
type(root.find('body').find('items'))

xml.etree.ElementTree.Element

In [51]:
child = root.find('body').find('items')[0]
child

<Element 'item' at 0x00000161C4B13630>

In [52]:
child.findall('*')#[0]

[<Element '거래금액' at 0x00000161C4B13680>,
 <Element '거래유형' at 0x00000161C4B136D0>,
 <Element '건축년도' at 0x00000161C4B13720>,
 <Element '년' at 0x00000161C4B137C0>,
 <Element '동' at 0x00000161C4B13860>,
 <Element '등기일자' at 0x00000161C4B138B0>,
 <Element '매도자' at 0x00000161C4B13900>,
 <Element '매수자' at 0x00000161C4B13950>,
 <Element '법정동' at 0x00000161C4B139A0>,
 <Element '아파트' at 0x00000161C4B139F0>,
 <Element '월' at 0x00000161C4B13A90>,
 <Element '일' at 0x00000161C4B13B30>,
 <Element '전용면적' at 0x00000161C4B13B80>,
 <Element '중개사소재지' at 0x00000161C4B13BD0>,
 <Element '지번' at 0x00000161C4B13C20>,
 <Element '지역코드' at 0x00000161C4B13C70>,
 <Element '층' at 0x00000161C4B13D10>,
 <Element '해제사유발생일' at 0x00000161C4B13D60>,
 <Element '해제여부' at 0x00000161C4B13DB0>]

In [53]:
element = child.findall('*')[0]

In [54]:
element.tag.strip()

'거래금액'

In [55]:
element.text.strip()

'197,000'

In [56]:
data = {}

In [57]:
tag = element.tag.strip()
text = element.text.strip()
data[tag] = text

In [58]:
data

{'거래금액': '197,000'}

In [59]:
def find_xml_items(response):
    root = ET.fromstring(response.content)
    item_list = []
    for child in root.find('body').find('items'):
        elements = child.findall('*')
        data = {}
        for element in elements:
            tag = element.tag.strip()
            text = element.text.strip()
            # print tag, text
            data[tag] = text
        item_list.append(data)  
    return item_list

In [60]:
items_list = find_xml_items(response)
items_list

[{'거래금액': '197,000',
  '거래유형': '중개거래',
  '건축년도': '2005',
  '년': '2024',
  '동': '',
  '등기일자': '',
  '매도자': '개인',
  '매수자': '개인',
  '법정동': '역삼동',
  '아파트': 'e-편한세상',
  '월': '3',
  '일': '3',
  '전용면적': '59.606',
  '중개사소재지': '서울 강남구',
  '지번': '755-4',
  '지역코드': '11680',
  '층': '6',
  '해제사유발생일': '',
  '해제여부': ''},
 {'거래금액': '61,000',
  '거래유형': '직거래',
  '건축년도': '2003',
  '년': '2024',
  '동': '',
  '등기일자': '24.04.03',
  '매도자': '개인',
  '매수자': '개인',
  '법정동': '역삼동',
  '아파트': '노블스위트',
  '월': '3',
  '일': '5',
  '전용면적': '78.94',
  '중개사소재지': '',
  '지번': '747',
  '지역코드': '11680',
  '층': '4',
  '해제사유발생일': '',
  '해제여부': ''},
 {'거래금액': '198,000',
  '거래유형': '중개거래',
  '건축년도': '2005',
  '년': '2024',
  '동': '',
  '등기일자': '',
  '매도자': '개인',
  '매수자': '개인',
  '법정동': '역삼동',
  '아파트': 'e-편한세상',
  '월': '3',
  '일': '6',
  '전용면적': '59.606',
  '중개사소재지': '서울 강남구',
  '지번': '755-4',
  '지역코드': '11680',
  '층': '8',
  '해제사유발생일': '',
  '해제여부': ''},
 {'거래금액': '23,000',
  '거래유형': '직거래',
  '건축년도': '2001',
  '년': '2024',
  '동': '',

### 4. Result : Display & Save as file 

In [61]:
result = pd.DataFrame(items_list)

In [62]:
result

,거래금액,거래유형,건축년도,년,동,등기일자,매도자,매수자,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"197,000",중개거래,2005,2024,,,개인,개인,역삼동,e-편한세상,3,3,59.606,서울 강남구,755-4,11680,6,,
1,"61,000",직거래,2003,2024,,24.04.03,개인,개인,역삼동,노블스위트,3,5,78.94,,747,11680,4,,
2,"198,000",중개거래,2005,2024,,,개인,개인,역삼동,e-편한세상,3,6,59.606,서울 강남구,755-4,11680,8,,
3,"23,000",직거래,2001,2024,,24.03.27,개인,개인,역삼동,한화진넥스빌,3,9,39.2,,706-20,11680,22,,
4,"191,000",중개거래,1998,2024,,,개인,개인,역삼동,동부센트레빌,3,9,84.97,서울 강남구,716,11680,2,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
201,"242,000",중개거래,2004,2024,,,개인,개인,도곡동,하이페리온,3,23,184.31,서울 강남구,968,11680,10,,
202,"274,000",중개거래,2006,2024,,,개인,개인,도곡동,도곡렉슬,3,28,84.9231,서울 강남구,527,11680,4,,
203,"208,000",중개거래,1978,2024,,,개인,개인,도곡동,도곡쌍용예가,3,29,107.53,서울 강남구,933,11680,5,,
204,"300,000",중개거래,2003,2024,,,개인,개인,도곡동,타워팰리스2,3,30,115.63,서울 강남구,467-17,11680,20,,


In [63]:
result.shape

(206, 19)

In [64]:
result.head()

,거래금액,거래유형,건축년도,년,동,등기일자,매도자,매수자,법정동,아파트,월,일,전용면적,중개사소재지,지번,지역코드,층,해제사유발생일,해제여부
0,"197,000",중개거래,2005,2024,,,개인,개인,역삼동,e-편한세상,3,3,59.606,서울 강남구,755-4,11680,6,,
1,"61,000",직거래,2003,2024,,24.04.03,개인,개인,역삼동,노블스위트,3,5,78.94,,747,11680,4,,
2,"198,000",중개거래,2005,2024,,,개인,개인,역삼동,e-편한세상,3,6,59.606,서울 강남구,755-4,11680,8,,
3,"23,000",직거래,2001,2024,,24.03.27,개인,개인,역삼동,한화진넥스빌,3,9,39.2,,706-20,11680,22,,
4,"191,000",중개거래,1998,2024,,,개인,개인,역삼동,동부센트레빌,3,9,84.97,서울 강남구,716,11680,2,,


#### : file name

In [65]:
name_info = in_lawd + "_" + in_deal_ymd
name_info

'강남구_202403'

In [66]:
file_name = f"data/result_{name_info}.csv"
file_name

'data/result_강남구_202403.csv'

In [67]:
result.to_csv(file_name, index=False, encoding='cp949')

## Appendix : Function 

In [68]:
def deal_by_lawd(cd, ymd):
    
    svckey = my_key
    
    area_cd = "LAWD_CD=" + cd
    deal_dt = "DEAL_YMD=" + ymd
    svc_key = "serviceKey=" + svckey
    url = base_url + "?" + area_cd + "&" + deal_dt + "&" + svc_key

    response = requests.get(url)
    items_list = find_xml_items(response)
    result = pd.DataFrame(items_list)
    
    name_info = in_lawd + "_" + in_deal_ymd
    file_name = f"data/result_{name_info}.csv"

    result.to_csv(file_name, index=False, encoding='cp949')
    
    return file_name

In [69]:
#### - Input Data : LAWD_CD, DEAL_YMD, serviceKey  입력받기
in_lawd = input("검색지역 입력 : ")
print(in_lawd)

in_lawd_cd = find_lawd_cd(in_lawd)
print(in_lawd_cd)

in_deal_ymd = input("계약월 입력(예: 201911) : ")
print(in_deal_ymd)

검색지역 입력 : 중구
중구
11140
계약월 입력(예: 201911) : 202403
202403


In [70]:
deal_by_lawd(in_lawd_cd, in_deal_ymd)

'data/result_중구_202403.csv'

### 추가 실습 :
- 공공API 데이터 끌어오기 친숙해지기
- **(필수) 특정월에 서울시 전체 구의 데이터를 한번에 가져오기(입력값: 연월 202402)**
- (선택) 특정 서울시 하나의 구의 과거 1년치 데이터를 한번에 가져오기(입력값: 특정구 하나) 

- 기상청_중기예보 조회서비스 (추가 API 사용신청 필요)    
https://www.data.go.kr/tcs/dss/selectApiDataDetailView.do?publicDataPk=15059468

In [106]:
import requests

url = 'http://apis.data.go.kr/1360000/MidFcstInfoService/getMidFcst'
params ={'serviceKey' : my_key, 'pageNo' : '1', 'numOfRows' : '10', 'dataType' : 'XML', 'stnId' : '108', 'tmFc' : '202405020600' }

response = requests.get(url, params=params)
#print(response.content)

In [107]:
print(response.text)

<OpenAPI_ServiceResponse>
	<cmmMsgHeader>
		<errMsg>SERVICE ERROR</errMsg>
		<returnAuthMsg>SERVICE_KEY_IS_NOT_REGISTERED_ERROR</returnAuthMsg>
		<returnReasonCode>30</returnReasonCode>
	</cmmMsgHeader>
</OpenAPI_ServiceResponse>


### 코드리뷰 과제(1)

- 서울시 모든 구의 리스트 추려내는 방법

In [144]:
lawd_cd.head()

,code,name,check
0,1100000000,서울특별시,존재
1,1111000000,서울특별시 종로구,존재
2,1111010100,서울특별시 종로구 청운동,존재
3,1111010200,서울특별시 종로구 신교동,존재
4,1111010300,서울특별시 종로구 궁정동,존재


- 파이썬으로 구하는 방법

In [133]:
tmp = df[df['name'].str.contains('서울특별시')]

In [135]:
#list(tmp['name'])

In [142]:
##### 코드리뷰 대상입니다 ####
name_list = list(tmp['name'])
goo_name = []
for name in name_list:
    if name[-1] == "구":
        name_split = name.split(' ')
        goo_name.append(name_split[1])
print(goo_name)
len(goo_name)

['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구', '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구', '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구']


25

- 판다스로 구하는 방법

In [137]:
df = lawd_cd

In [140]:
#df[df['name'].str.contains('서울특별시')]['name'].str.split().str[1].dropna().unique()

In [139]:
##### 코드리뷰 대상입니다 ####
seoul_gu_list = df[df['name'].str.contains('서울특별시')]['name'].str.split().str[1].dropna().unique()
seoul_gu_list

array(['종로구', '중구', '용산구', '성동구', '광진구', '동대문구', '중랑구', '성북구', '강북구',
       '도봉구', '노원구', '은평구', '서대문구', '마포구', '양천구', '강서구', '구로구', '금천구',
       '영등포구', '동작구', '관악구', '서초구', '강남구', '송파구', '강동구'], dtype=object)

In [147]:
seoul_gu_list = list(set(seoul_gu_list))
len(seoul_gu_list)

25

- **(필수) 특정월에 서울시 전체 구의 데이터를 한번에 가져오기(입력값: 연월 202402)**

In [149]:
#seoul_gu_list

In [152]:
for gu in seoul_gu_list:
    deal_by_lawd(gu, '202402')